                                                          Car Resale Data Analysis Project

Objective: The objective of this project is to analyze car resale patterns and factors influencing resale prices in India. This will be achieved through data analysis of the 'car resale' dataset sourced from Kaggle, focusing on transactions that occurred in the year 2023.

Dataset Link: https://www.kaggle.com/datasets/rahulmenon1758/car-resale-prices

Tools Used: AWS s3 (As an external storage system),Databricks (For data cleaning,transformation and visualization),Pyspark (Programming language)

In [0]:
dbutils.fs.unmount('/mnt/Car_DA_Project')

In [0]:
access_key = 'Your Access Key'      
secret_key = 'Your Secret Access Key'
encoded_secret_key = secret_key.replace("/", "%2F")
aws_bucket_name = "car-resale-data-analysis"
mount_name = "Car_DA_Project"

dbutils.fs.mount(f"s3a://{access_key}:{encoded_secret_key}@{aws_bucket_name}", f"/mnt/{mount_name}")
display(dbutils.fs.ls(f"/mnt/{mount_name}"))


In [0]:
df = spark.read.option("header", True).csv(
    "dbfs:/mnt/Car_DA_Project/dataset/car_resale_prices.csv"
)
display(df)

Data Cleaning and Transformation.

In [0]:
# Print df schema.
display(df.printSchema())

In [0]:
# Get the shape of df.

from pyspark.sql.functions import *
print(df.count() , len(df.columns))  # df.count() - to get total rows , len(df.columns) - to get total columns.


In [0]:
df = df.drop("_c0")  # Drop '_c0' column.

In [0]:
df = (df.dropDuplicates())  # df.dropduplicates() removes all duplicates from the df and returns a new df with all unique rows.
display(df.count())  # Count of duplicate rows = Total rows - count of unique rows = 17446 - 17243 = 339 duplicate rows

In [0]:
# Rename 'full_name' column.
df = df.withColumnRenamed("full_name" , "car_model")

# Remove registration year information from the column values.
df = df.withColumn("car_model", expr("substring(car_model, 6)"))  # Trims string value

display(df)

In [0]:
df = df.withColumn("car_company",expr("split(car_model,' ')[0]"))
display(df)

In [0]:
# Create new column that contains only numerical price information from the 'resale_price' column.
df = df.withColumn('resale_price_in_Lakh_Rupees',expr("split(resale_price,' ')[1]"))  # splits the string based on white space.
df=df.drop('resale_price')
display(df)

In [0]:
# Remove 'cc' substring from the 'engine_capacity' column.
df = df.withColumn('engine_capacity_in_cc' , expr("split(engine_capacity,' ')[0]"))
df=df.drop('engine_capacity')
display(df)

In [0]:
# Create new column that contains only km digits from 'kms_driven' column.
df = df.withColumn('Kms_driven' , expr("split(kms_driven,' ')[0]"))
df = df.withColumn('Kms_driven' , regexp_replace('Kms_driven',',',''))  # replace ',' with void from Kms_driven.
display(df)

In [0]:
# Transform 'mileage' column.
df = df.withColumn('mileage_in_kmpl' , expr("split(mileage,' ')[0]"))
df = df.drop("mileage")    
display(df)

In [0]:
# Transform 'max_power' column.
df = df.withColumn("max_power_in_bhp", expr("substring(max_power, 1, locate('bhp', max_power) - 1)"))
df = df.drop('max_power')
display(df)

In [0]:
# replace available values.
df = df.withColumn("insurance",regexp_replace('insurance','Not Available','Without Insuarance')).withColumn("insurance",regexp_replace('insurance','1','First Party Insuarance')).withColumn("insurance",regexp_replace('insurance','2','Second Party Insuarance'))
display(df)

In [0]:
# Extract only year information.
df = df.withColumn("registered_yr" ,expr("substring(registered_year,-4)"))
df = df.drop("registered_year")
display(df)

In [0]:
# Rearrange columns in the df.
df = df.select('car_company','car_model','registered_yr','Kms_driven','owner_type','body_type','seats','transmission_type','fuel_type','engine_capacity_in_cc','mileage_in_kmpl','max_power_in_bhp','insurance','city','resale_price_in_Lakh_Rupees')
display(df)

In [0]:
from pyspark.sql.types import IntegerType,DoubleType

# Change columns data type.

df = df.withColumn("registered_yr",df["registered_yr"].cast(IntegerType())).withColumn("seats",df["seats"].cast(IntegerType())).withColumn("engine_capacity_in_cc",df["engine_capacity_in_cc"].cast(IntegerType())).withColumn("mileage_in_kmpl",df["mileage_in_kmpl"].cast(IntegerType())).withColumn("max_power_in_bhp",df["max_power_in_bhp"].cast(IntegerType())).withColumn("resale_price_in_Lakh_Rupees",df["resale_price_in_Lakh_Rupees"].cast(DoubleType())).withColumn("Kms_driven",df["Kms_driven"].cast(DoubleType()))  

display(df.printSchema())

root
 |-- car_company: string (nullable = true)
 |-- car_model: string (nullable = true)
 |-- registered_yr: integer (nullable = true)
 |-- Kms_driven: double (nullable = true)
 |-- owner_type: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- seats: integer (nullable = true)
 |-- transmission_type: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- engine_capacity_in_cc: integer (nullable = true)
 |-- mileage_in_kmpl: integer (nullable = true)
 |-- max_power_in_bhp: integer (nullable = true)
 |-- insurance: string (nullable = true)
 |-- city: string (nullable = true)
 |-- resale_price_in_Lakh_Rupees: double (nullable = true)



Data Profiling

In [0]:
from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show() 

# df.na.drop() To drop null rows.

+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+---------------------------+
|car_company|car_model|registered_yr|Kms_driven|owner_type|body_type|seats|transmission_type|fuel_type|engine_capacity_in_cc|mileage_in_kmpl|max_power_in_bhp|insurance|city|resale_price_in_Lakh_Rupees|
+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+---------------------------+
|          0|        0|           68|         3|        45|        0|   10|                0|        0|                   13|            502|             756|        7|   0|                        198|
+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+-------------------

- For 'max_power_in_bhp'  and  'mileage_in_kmpl' columns, the intention is to populate null entries with randomly generated values drawn from a range between the 25th and 75th percentiles. The aim is to distribute these values more evenly across the dataset, as opposed to utilizing constant values like mean or median.
- Conversely, for the remaining columns, null values can be filled using mean or median values. The rationale behind this decision lies in the relatively lower quantity of missing data in these columns, thus making the impact on data quality less significant.

In [0]:
min_power = 78     # This vales are selected based on the understanding of value distribution on the registered_yr column.
max_power = 118

df = df.withColumn('max_power_in_bhp', when(col('max_power_in_bhp').isNull(),(rand() * (max_power - min_power) + min_power).cast('int')).otherwise(col('max_power_in_bhp')))
display(df.where(col('max_power_in_bhp').isNull()).count())


0

In [0]:
min_mileage = 6
max_mileage = 21

df = df.withColumn('mileage_in_kmpl', when(col('mileage_in_kmpl').isNull(),(rand() * (max_mileage - min_mileage) + min_mileage).cast('int')).otherwise(col('mileage_in_kmpl')))

display(df.where(col('mileage_in_kmpl').isNull()).count())

0

In [0]:
# Handle nulls in resale_price_in_Lakh_Rupees,owner_type,seats,engine_capacity_in_cc,insurance at once using mean value.

df = df.na.fill({'registered_yr': 2017,'Kms_driven': 58597,'owner_type': 'Third Owner','seats':5,'engine_capacity_in_cc':1422,'insurance':'Third Party','resale_price_in_Lakh_Rupees':8.71})


In [0]:
# Re-check null values in df.

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+---------------------------+
|car_company|car_model|registered_yr|Kms_driven|owner_type|body_type|seats|transmission_type|fuel_type|engine_capacity_in_cc|mileage_in_kmpl|max_power_in_bhp|insurance|city|resale_price_in_Lakh_Rupees|
+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+---------------------------+
|          0|        0|            0|         0|         0|        0|    0|                0|        0|                    0|              0|               0|        0|   0|                          0|
+-----------+---------+-------------+----------+----------+---------+-----+-----------------+---------+---------------------+---------------+----------------+---------+----+-------------------

Write a cleaned dataframe to extrenal storage location.

In [0]:
# write a cleaned df to a csv file.
df.write.mode("overwrite").option("header" , True).option("inferschema" , True).csv('dbfs:/mnt/Car_DA_Project/Cleaned_dataset/')

# Save dataframe to table for querying using SQL
df.write.option("overwriteSchema", True).mode("overwrite").saveAsTable("default.cleaned_data")

Statistical Summary

In [0]:
summary = df.summary() 
display(summary)

# Export statistical summary.
summary.write.mode("overwrite").option("header",True).csv('dbfs:/mnt/Car_DA_Project/Dataframe_Suumary/')

summary,car_company,car_model,registered_yr,Kms_driven,owner_type,body_type,seats,transmission_type,fuel_type,engine_capacity_in_cc,mileage_in_kmpl,max_power_in_bhp,insurance,city,resale_price_in_Lakh_Rupees
count,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243,17243
mean,null,null,2016.4249840514992,58596.87873339906,null,null,5.203792843472714,null,null,1422.925303021516,18.799918807632082,103.20680855999537,null,null,8.712372556979586
stddev,null,null,3.6454619954598093,64504.80134338061,null,null,0.6660034570513632,null,null,473.8774757981255,4.44531217853078,44.025029868041955,null,null,9.645090590535398
min,Audi,Audi A3 35 TDI Attraction,2002,286.0,Fifth Owner,Audi,2,Automatic,CNG,0,6,25,Comprehensive,Agra,1.0
25%,null,null,2014,32000.0,null,null,5,null,null,1197,17,78,null,null,3.89
50%,null,null,2017,54826.0,null,null,5,null,null,1248,18,88,null,null,5.95
75%,null,null,2019,79455.0,null,null,5,null,null,1498,21,117,null,null,9.08
max,Volvo,Volvo XC60 Inscription D5 BSIV,2023,6275000.0,Third Owner,Wagon,14,Manual,Petrol,5998,140,558,Zero Dep,Pune,97.0


Data Analysis

In [0]:
%sql
SELECT 
    car_company, 
    COUNT(car_model) AS Total_cars_sold_per_company
FROM 
    default.cleaned_data
GROUP BY 
    car_company
ORDER BY 
    Total_cars_sold_per_company DESC;

-- In 2023, Maruti, Hyundai, and Honda accounted for approximately 60% of the resold car market in India, totaling more than 10,000 vehicles. Conversely, Force, Opel Corsa, Hindustan, Daewoo, and Bentley experienced the lowest sales figures among car companies operating in the country during that year.

car_company,Total_cars_sold_per_company
Maruti,4766
Hyundai,3571
Honda,1777
Tata,989
Mahindra,790
Toyota,719
Renault,626
Ford,555
Volkswagen,551
Mercedes-Benz,469


In [0]:
%sql
WITH car_model_counts AS (
    SELECT 
        car_company, 
        car_model, 
        COUNT(car_model) AS count_car_model
    FROM 
        default.cleaned_data
    WHERE 
        car_company IN ('Maruti', 'Hyundai', 'Honda')
    GROUP BY 
        car_company, 
        car_model
),
ranked_car_models AS (
    SELECT 
        car_company, 
        car_model, 
        count_car_model,
        ROW_NUMBER() OVER (PARTITION BY car_company ORDER BY count_car_model DESC) AS row_number
    FROM 
        car_model_counts
)
SELECT 
    car_company, 
    car_model, 
    count_car_model
FROM 
    ranked_car_models
WHERE 
    row_number <= 5
ORDER BY 
    car_company, 
    count_car_model DESC;


car_company,car_model,count_car_model
Honda,Honda Brio S MT,81
Honda,Honda City i VTEC V,73
Honda,Honda WR-V i-VTEC VX,61
Honda,Honda City i VTEC CVT VX,58
Honda,Honda Amaze S i-Vtech,56
Hyundai,Hyundai Grand i10 Sportz,134
Hyundai,Hyundai i10 Magna,85
Hyundai,Hyundai i20 Sportz 1.2,85
Hyundai,Hyundai Grand i10 1.2 Kappa Sportz BSIV,69
Hyundai,Hyundai i20 Asta 1.2,69


In [0]:
%sql
-- Get the details of top 5 cars with  highest resale price in descing order for each dominating car model.
select
  *
from
  (
    SELECT
      car_model,
      registered_yr,
      Kms_driven,
      owner_type,
      insurance,
      city,
      resale_price_in_Lakh_Rupees,
      row_number() over (
        PARTITION BY car_model
        ORDER BY
          resale_price_in_Lakh_Rupees DESC
      ) AS resale_price_precedence
    FROM
      default.cleaned_data
    WHERE
      car_model in (
        'Maruti Swift VXI',
        'Maruti Alto 800 LXI',
        'Maruti Wagon R VXI BS IV',
        'Maruti Baleno 1.2 Delta',
        'Hyundai Grand i10 Sportz',
        'Maruti Swift Dzire VXI',
        'Maruti Alto K10 VXI',
        'Maruti Wagon R LXI CNG',
        'Hyundai i20 Sportz 1.2',
        'Hyundai i10 Magna',
        'Hyundai Grand i10 1.2 Kappa Sportz BSIV',
        'Hyundai i20 Asta 1.2',
        'Honda Amaze S i-Vtech',
        'Honda City i VTEC CVT VX',
        'Honda WR-V i-VTEC VX',
        'Honda City i VTEC V',
        'Honda Brio S MT'
      )
  )
where
  resale_price_precedence <= 5;
-- fetching only top 5 rows from each partitions.

car_model,registered_yr,Kms_driven,owner_type,insurance,city,resale_price_in_Lakh_Rupees,resale_price_precedence
Honda Amaze S i-Vtech,2015,31881.0,First Owner,Comprehensive,Bangalore,5.05,1
Honda Amaze S i-Vtech,2016,26985.0,First Owner,Third Party insurance,Ahmedabad,4.91,2
Honda Amaze S i-Vtech,2016,120757.0,First Owner,Comprehensive,Chennai,4.88,3
Honda Amaze S i-Vtech,2016,66655.0,First Owner,Comprehensive,Bangalore,4.82,4
Honda Amaze S i-Vtech,2015,74385.0,First Owner,Third Party,Hyderabad,4.58,5
Honda Brio S MT,2016,65453.0,First Owner,Comprehensive,Bangalore,4.38,1
Honda Brio S MT,2016,65394.0,First Owner,Third Party insurance,Bangalore,4.24,2
Honda Brio S MT,2015,22262.0,Second Owner,Comprehensive,Bangalore,4.22,3
Honda Brio S MT,2013,50000.0,Third Owner,Third Party insurance,Pune,4.2,4
Honda Brio S MT,2013,38197.0,First Owner,Third Party,Bangalore,4.06,5


In [0]:
%sql
SELECT 
    registered_yr, 
    COUNT(registered_yr) AS Number_of_cars_sold_per_registration_year
FROM 
    default.cleaned_data
GROUP BY 
    registered_yr
ORDER BY 
    Number_of_cars_sold_per_registration_year DESC;


registered_yr,Number_of_cars_sold_per_registration_year
2017,2031
2018,1973
2019,1681
2016,1647
2021,1366
2015,1362
2014,1265
2020,1174
2022,1013
2013,944


In [0]:
%sql
SELECT 
    owner_type, 
    COUNT(owner_type) AS Total_cars_sold_per_ownership_type
FROM 
    default.cleaned_data
GROUP BY 
    owner_type
ORDER BY 
    Total_cars_sold_per_ownership_type DESC;


owner_type,Total_cars_sold_per_ownership_type
First Owner,12177
Second Owner,4087
Third Owner,806
Fourth Owner,124
Fifth Owner,49


In [0]:
%sql
SELECT 
    insurance, 
    COUNT(insurance) AS Total_cars_sold_per_insurance_type
FROM 
    default.cleaned_data
GROUP BY 
    insurance
ORDER BY 
    Total_cars_sold_per_insurance_type DESC;


insurance,Total_cars_sold_per_insurance_type
Third Party insurance,7367
Comprehensive,6404
Third Party,1980
Zero Dep,834
Without Insuarance,650
First Party Insuarance,5
Second Party Insuarance,3


In [0]:
%sql
SELECT 
    city, 
    COUNT(city) AS Total_cars_sold_per_city
FROM 
    default.cleaned_data
GROUP BY 
    city
ORDER BY 
    Total_cars_sold_per_city DESC;


city,Total_cars_sold_per_city
Delhi,2996
Bangalore,2322
Mumbai,2082
Hyderabad,1552
Pune,1384
Chennai,1339
Ahmedabad,1315
Kolkata,1166
Gurgaon,1034
Jaipur,882


Understand Correlations between columns.

- registered_yr & resale_price_in_Lakh_Rupees = 0.33 = weak positive correlation
- kms_driven & resale_price_in_Lakh_Rupees = -0.07 = negative no 

-- There is no direct correlation between registration year or kms_driven with the resale price of the car,as it would be seen in the real world scenario.

In [0]:
# Let's understand how registration year impacts the re-sale price of a car?
display(df.stat.corr('registered_yr','resale_price_in_Lakh_Rupees'))

0.33888218643053675

In [0]:
%sql
WITH Temp_df AS (
    SELECT 
        car_model, 
        registered_yr, 
        kms_driven, 
        owner_type, 
        insurance, 
        city, 
        resale_price_in_Lakh_Rupees
    FROM 
        default.cleaned_data
    WHERE 
        car_model = 'Hyundai Grand i10 Sportz'  -- Specify your desired car model here.
),
Ranked_cars AS (
    SELECT 
        car_model, 
        registered_yr, 
        kms_driven, 
        owner_type, 
        insurance, 
        city, 
        resale_price_in_Lakh_Rupees,
        ROW_NUMBER() OVER (PARTITION BY city ORDER BY resale_price_in_Lakh_Rupees DESC) AS highest_re_sale_price_order
    FROM 
        Temp_df
)
SELECT 
    car_model, 
    registered_yr, 
    kms_driven, 
    owner_type, 
    insurance, 
    city, 
    resale_price_in_Lakh_Rupees
FROM 
    Ranked_cars
WHERE 
    highest_re_sale_price_order <= 5
ORDER BY 
    city, 
    highest_re_sale_price_order;

car_model,registered_yr,kms_driven,owner_type,insurance,city,resale_price_in_Lakh_Rupees
Hyundai Grand i10 Sportz,2015,30000.0,First Owner,Third Party insurance,Agra,4.15
Hyundai Grand i10 Sportz,2015,60000.0,Second Owner,Third Party insurance,Agra,3.7
Hyundai Grand i10 Sportz,2014,70000.0,Third Owner,Third Party insurance,Agra,3.0
Hyundai Grand i10 Sportz,2018,48000.0,First Owner,Comprehensive,Ahmedabad,5.25
Hyundai Grand i10 Sportz,2016,36493.0,First Owner,Comprehensive,Ahmedabad,4.54
Hyundai Grand i10 Sportz,2015,72036.0,Second Owner,Third Party insurance,Ahmedabad,4.3
Hyundai Grand i10 Sportz,2015,52596.0,Second Owner,Comprehensive,Ahmedabad,4.28
Hyundai Grand i10 Sportz,2016,53162.0,First Owner,Third Party,Ahmedabad,4.26
Hyundai Grand i10 Sportz,2017,66000.0,First Owner,Comprehensive,Bangalore,5.5
Hyundai Grand i10 Sportz,2016,25960.0,First Owner,Comprehensive,Bangalore,5.06
